In [1]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd
import requests
import pymongo
from flask import Flask
#flask_pymongo import is generating an error code. Pip install Flask-Pymongo
##documentation here: https://flask-pymongo.readthedocs.io/en/latest/
from flask_pymongo import PyMongo
from flask import render_template, redirect
###pip install fixed the error.

In [4]:
#Set up the executable path and the browser. Boiler plate code.
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False) 

In [5]:
#link to the url from the assignment readme
redplanet_url = 'https://redplanetscience.com/'
browser.visit(redplanet_url)

In [16]:
#create the html, soup, and do the scraping
html = browser.html

redplanet_soup = bs(html, 'html.parser')

#assign as variables the scraped data
news_title = redplanet_soup.find_all('div', class_='content_title')[0].text
news_paragraph = redplanet_soup.find_all('div', class_='article_teaser_body')[0].text

print(news_title)
print("=================================================")
print(news_paragraph)

Space History Is Made in This NASA Robot Factory
From rockets to rovers, JPL's Spacecraft Assembly Facility has been at the center of robotic spaceflight. Here's a closer look at what makes it so special.


In [59]:
# JPL Mars Space Images - Featured Image
##Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).
###Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.
####Make sure to find the image url to the full size `.jpg` image.
#####Make sure to save a complete url string for this image.

jpl_image_url = 'https://spaceimages-mars.com'
image_url = 'https://spaceimages-mars.com/image/?search=&category=Mars'

browser.visit(image_url)

#browser.find_by_name('featured').click()

html = browser.html

image_soup = bs(html, 'html.parser')

image_path = [i.get('src') for i in image_soup.find_all('img', class_='headerimage fade-in')]
image_path[0]
              


IndexError: list index out of range

In [58]:
featured_image_url = jpl_image_url + image_path[0]
featured_image_url

IndexError: list index out of range

In [25]:
#Mars Facts
##Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
###Use Pandas to convert the data to a HTML table string.

facts_url = 'https://galaxyfacts-mars.com'
browser.visit(facts_url)


In [28]:
#Mars Facts Pandas Portion
facts_df = pd.read_html(facts_url)[1]
facts_df.columns = ['Description', 'Mars']
facts_df

,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [34]:
#Mars Facts pandas dataframe to html portion
##transpose the df. Documentation: https://numpy.org/doc/stable/reference/generated/numpy.matrix.transpose.html

facts_transpose = facts_df.transpose()
facts_transpose

,0,1,2,3,4,5,6,7,8
Description,Equatorial Diameter:,Polar Diameter:,Mass:,Moons:,Orbit Distance:,Orbit Period:,Surface Temperature:,First Record:,Recorded By:
Mars,"6,792 km","6,752 km",6.39 × 10^23 kg (0.11 Earths),2 ( Phobos & Deimos ),"227,943,824 km (1.38 AU)",687 days (1.9 years),-87 to -5 °C,2nd millennium BC,Egyptian astronomers


In [37]:
#take the transposed dataframe and render in in html with .to_html
facts_html = facts_transpose.to_html('MarsFacts.html', index=False)
facts_html

In [43]:
#Mars Hemispheres
#Visit the astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of Mar's hemispheres.
##You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
###Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. 
####Use a Python dictionary to store the data using the keys `img_url` and `title`.
####Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

#####First set up the URLs and paths and soup

hemi_url = 'https://marshemispheres.com/'
hemi_url_path = requests.get(hemi_url)

hemi_soup = bs(hemi_url_path.content)


In [44]:
#Next find the div items for the images
hemi_div_item = hemi_soup.find_all('div', class_='item')

In [45]:
#Next find the links to the images and append them to a dictionary
hemi_links = []
for i in hemi_div_item:
    for a in i.find_all('a', href=True):
        if a.text:
            hemi_links.append(a['href'])
#print(hemi_links)

['cerberus.html', 'schiaparelli.html', 'syrtis.html', 'valles.html']


In [47]:
#NEXT take the dictionary of individual links and combine it with the url for the hemishperes
full_hemi_links = [hemi_url + i for i in hemi_links]
full_hemi_links

['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']